In [2]:
# Merging datasets
import os
import sys
import numpy as np
from tqdm import tqdm

from osgeo import gdal
from osgeo import osr

def read_img(filename, with_data=True):
    dataset = gdal.Open(filename)  # 打开文件
    im_width = dataset.RasterXSize  # 栅格矩阵的列数
    im_height = dataset.RasterYSize  # 栅格矩阵的行数
    im_geotrans = dataset.GetGeoTransform()  # 仿射矩阵
    im_proj = dataset.GetProjection()  # 地图投影信息
    if not with_data:
        return dataset, im_proj, im_geotrans, im_height, im_width
    else:
        im_data = dataset.ReadAsArray(0, 0, im_width, im_height)  # 将数据写成数组，对应栅格矩阵
        return dataset, im_proj, im_geotrans, im_height, im_width, im_data

def write_img(filename, im_proj, im_geotrans, im_data):
    # gdal数据类型包括
    # gdal.GDT_Byte,
    # gdal .GDT_UInt16, gdal.GDT_Int16, gdal.GDT_UInt32, gdal.GDT_Int32,
    # gdal.GDT_Float32, gdal.GDT_Float64
    
    # 判断栅格数据的数据类型
    if 'int8' in im_data.dtype.name:
        datatype = gdal.GDT_Byte
    elif 'int16' in im_data.dtype.name:
        datatype = gdal.GDT_UInt16
    else:
        datatype = gdal.GDT_Float32

    # 判读数组维数
    if len(im_data.shape) == 3:
        im_bands, im_height, im_width = im_data.shape
    else:
        im_bands, (im_height, im_width) = 1, im_data.shape

    # 创建文件
    driver = gdal.GetDriverByName("GTiff")  # 数据类型必须有，因为要计算需要多大内存空间
    dataset = driver.Create(filename, im_width, im_height, im_bands, datatype)

    dataset.SetGeoTransform(im_geotrans)  # 写入仿射变换参数
    dataset.SetProjection(im_proj)  # 写入投影

    if im_bands == 1:
        dataset.GetRasterBand(1).WriteArray(im_data)  # 写入数组数据
    else:
        for i in range(im_bands):
            dataset.GetRasterBand(i + 1).WriteArray(im_data[i])
    del dataset

def make_merge(rgb_path, sar_path, merge_path):
    ds_sar, im_proj_sar, im_geotrans_sar, height_sar, width_sar, data_sar = read_img(sar_path, True)
    ds_opt, im_proj_opt, im_geotrans_opt, height_opt, width_opt, data_opt = read_img(rgb_path, True)
    data_sar_cat = data_sar[np.newaxis, :, :]
    data_merge = np.concatenate([data_opt, data_sar_cat], axis=0)
    write_img(merge_path, im_proj_opt, im_geotrans_opt, data_merge)



In [33]:
if __name__ == '__main__':
    root = sys.argv[1]
    root = r'C:\Users\saada\Desktop\Drive\UBS\Workshop_Contest\data\track2/'
    sets = ['train', 'val', 'test']
    for set in sets:
        set_dir = os.path.join(root, set)
        rgb_dir = os.path.join(set_dir, 'rgb')
        sar_dir = os.path.join(set_dir, 'sar')
        merge_dir = os.path.join(set_dir, 'merge')
        if not os.path.exists(merge_dir):
            os.mkdir(merge_dir)
        for f in tqdm(os.listdir(rgb_dir)):
            if f.endswith('.tif'):
                rgb_path = os.path.join(rgb_dir, f)
                sar_path = os.path.join(sar_dir, f)
                merge_path = os.path.join(merge_dir, f)
                make_merge(rgb_path, sar_path, merge_path)

100%|████████████████████████████████████████████████████████████████████████████████| 579/579 [00:27<00:00, 21.32it/s]


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\saada\\Desktop\\Drive\\UBS\\Workshop_Contest\\data\\track2/test\\merge'

In [5]:
if __name__ == '__main__':
    root = sys.argv[1]
    root = r'C:\Users\saada\Desktop\Drive\UBS\Workshop_Contest\track2\HeightEstimation\workspace\dataset\data/'
    sets = [ 'track2_test_data']
    for set in sets:
        set_dir = os.path.join(root, set)
        rgb_dir = os.path.join(set_dir, 'rgb')
        sar_dir = os.path.join(set_dir, 'sar')
        merge_dir = os.path.join(set_dir, 'merge')
        if not os.path.exists(merge_dir):
            os.mkdir(merge_dir)
        for f in tqdm(os.listdir(rgb_dir)):
            if f.endswith('.tif'):
                rgb_path = os.path.join(rgb_dir, f)
                sar_path = os.path.join(sar_dir, f)
                merge_path = os.path.join(merge_dir, f)
                make_merge(rgb_path, sar_path, merge_path)

100%|████████████████████████████████████████████████████████████████████████████████| 605/605 [00:26<00:00, 22.96it/s]


In [34]:
import torch
import torch.nn as nn

from functools import partial
from torch.nn import functional as F

affine_par = True

def conv3x3(in_planes,
            out_planes,
            stride=1):
    "3x3 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

class PSPModule(nn.Module):
    """
    Reference:s
        Zhao, Hengshuang, et al. *"Pyramid scene parsing network."*
    """
    def __init__(self, features, out_features=512, sizes=(1, 2, 3, 6)):
        super(PSPModule, self).__init__()

        self.stages = []
        self.stages = nn.ModuleList([self._make_stage(features, out_features, size) for size in sizes])
        self.bottleneck = nn.Sequential(
            nn.Conv2d(features+len(sizes)*out_features, out_features, kernel_size=3, padding=1, dilation=1, bias=False),
            nn.BatchNorm2d(out_features),
            nn.Dropout2d(0.1)
            )

    def _make_stage(self, features, out_features, size):
        prior = nn.AdaptiveAvgPool2d(output_size=(size, size))
        conv = nn.Conv2d(features, out_features, kernel_size=1, bias=False)
        bn = nn.BatchNorm2d(out_features)
        return nn.Sequential(prior, conv, bn)

    def forward(self, feats):
        h, w = feats.size(2), feats.size(3)
        priors = [F.upsample(input=stage(feats), size=(h, w), mode='bilinear', align_corners=True) for stage in self.stages] + [feats]
        bottle = self.bottleneck(torch.cat(priors, 1))
        return bottle

class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self,
                 inplanes,
                 planes,
                 stride=1,
                 dilation=1,
                 downsample=None,
                 fist_dilation=1,
                 multi_grid=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=dilation*multi_grid, dilation=dilation*multi_grid, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=False)
        self.relu_inplace = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.dilation = dilation
        self.stride = stride

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out = out + residual
        out = self.relu_inplace(out)
        return out


class ASPPModule(nn.Module):
    """
    Reference: 
        Chen, Liang-Chieh, et al. *"Rethinking Atrous Convolution for Semantic Image Segmentation."*
    """
    def __init__(self,
                 features,
                 inner_features=256,
                 out_features=512,
                 dilations=(12, 24, 36)):
        super(ASPPModule, self).__init__()

        self.conv1 = nn.Sequential(nn.AdaptiveAvgPool2d((1,1)),
                                   nn.Conv2d(features, inner_features, kernel_size=1, padding=0, dilation=1, bias=False),
                                   nn.BatchNorm2d(inner_features))
        self.conv2 = nn.Sequential(nn.Conv2d(features, inner_features, kernel_size=1, padding=0, dilation=1, bias=False),
                                   nn.BatchNorm2d(inner_features))
        self.conv3 = nn.Sequential(nn.Conv2d(features, inner_features, kernel_size=3, padding=dilations[0], dilation=dilations[0], bias=False),
                                   nn.BatchNorm2d(inner_features))
        self.conv4 = nn.Sequential(nn.Conv2d(features, inner_features, kernel_size=3, padding=dilations[1], dilation=dilations[1], bias=False),
                                   nn.BatchNorm2d(inner_features))
        self.conv5 = nn.Sequential(nn.Conv2d(features, inner_features, kernel_size=3, padding=dilations[2], dilation=dilations[2], bias=False),
                                   nn.BatchNorm2d(inner_features))

        self.bottleneck = nn.Sequential(
            nn.Conv2d(inner_features * 5, out_features, kernel_size=1, padding=0, dilation=1, bias=False),
            nn.BatchNorm2d(out_features),
            nn.Dropout2d(0.1)
            )
        
    def forward(self, x):
        _, _, h, w = x.size()
        feat1 = F.interpolate(self.conv1(x), size=(h, w), mode='bilinear', align_corners=True)
        feat2 = self.conv2(x)
        feat3 = self.conv3(x)
        feat4 = self.conv4(x)
        feat5 = self.conv5(x)
        out = torch.cat((feat1, feat2, feat3, feat4, feat5), 1)

        bottle = self.bottleneck(out)
        return bottle

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes):
        self.inplanes = 128
        super(ResNet, self).__init__()
        self.conv1 = conv3x3(3, 64, stride=2)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU(inplace=False)
        self.conv2 = conv3x3(64, 64)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU(inplace=False)
        self.conv3 = conv3x3(64, 128)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU(inplace=False)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.relu = nn.ReLU(inplace=False)

        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=1, dilation=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=1, dilation=4, multi_grid=(1,2,4))
        # self.layer4 = self._make_layer(block, 512, layers[3], stride=1, dilation=4, multi_grid=(1,1,1))

        self.head = nn.Sequential(nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
                                  nn.BatchNorm2d(256),
                                  nn.Dropout2d(0.1),
                                  nn.Conv2d(256, 1, kernel_size=1, stride=1, padding=0, bias=True),
                                  nn.Sigmoid())

        self.ppm = PSPModule(2048, 256)

    def forward(self, x):
        x = self.relu1(self.bn1(self.conv1(x))) # (64, 256, 256)
        x = self.relu2(self.bn2(self.conv2(x))) # (64, 256, 256)
        x = self.relu3(self.bn3(self.conv3(x))) # (128, 256, 256)
        x0 = self.maxpool(x) # (128, 128, 128)

        x1 = self.layer1(x0)  # (256, 128, 128)
        x2 = self.layer2(x1) # (512, 64, 64)
        x3 = self.layer3(x2) # (1024, 64, 64)
        x4 = self.layer4(x3) # (2048, 64, 64)

        conv_out = [x1, x2, x3, x4]
        psp_out = self.ppm(x4)
        f = psp_out
        x = self.head(f)
        return [x]

    def _make_layer(self, block, planes, blocks, stride=1, dilation=1, multi_grid=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion,affine = affine_par))

        layers = []
        generate_multi_grid = lambda index, grids: grids[index%len(grids)] if isinstance(grids, tuple) else 1
        layers.append(block(self.inplanes, planes, stride,dilation=dilation, downsample=downsample, multi_grid=generate_multi_grid(0, multi_grid)))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, dilation=dilation, multi_grid=generate_multi_grid(i, multi_grid)))

        return nn.Sequential(*layers)


# def Res_Deeplab(num_classes=6):
#     model = ResNet(Bottleneck,[3, 4, 23, 3], num_classes)
#     return model


def Res_Deeplab(num_classes=6):
    model = ResNet(Bottleneck,[3, 4, 6, 3], num_classes)
    return model


C:\Users\saada\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
import os
import sys
import argparse
import os.path as osp
import torch.optim as optim
import torch.backends.cudnn as cudnn

from tqdm import tqdm
from torch.utils import data
from torch.nn import functional as F
#C:\Users\saada\Desktop\Drive\UBS\Workshop_Contest\track2\HeightEstimation\utils
from utils.eval_utils import *
from networks.baseline import Res_Deeplab
from dataset.dataset import ISPRSDataSet, ISPRSDataValSet

from utils.criterion import CriterionDSN
from utils.metrics import AverageMeter, Result
from utils.encoding import DataParallelModel, DataParallelCriterion

torch_ver = torch.__version__[:3]

if torch_ver == '0.3':
    from torch.autograd import Variable

IGNORE_LABEL = 255
IMG_MEAN = np.array((120.47595769, 81.79931481, 81.19268267), dtype=np.float32)


DATA_DIRECTORY = 'workspace/dataset/data'
DATA_LIST_PATH = 'workspace/dataset/train.txt'
TEST_DATA_LIST_PATH = 'workspace/dataset/train.txt'
'''
DATA_DIRECTORY = '/workspace/dataset/data'
DATA_LIST_PATH = '/workspace/dataset/train.txt'
TEST_DATA_LIST_PATH = '/workspace/dataset/test.txt'
'''

BATCH_SIZE = 6
LEARNING_RATE = 1e-2
WEIGHT_DECAY = 0.0005

GPU_NUM = '0'
INPUT_SIZE = '512,512'

POWER = 0.9
MOMENTUM = 0.9
NUM_CLASSES = 1
NUM_STEPS = 80000
RANDOM_SEED = 111

SAVE_NUM_IMAGES = 2
SAVE_PRED_EVERY = 5000

RESTORE_FROM = '/workspace/pretrain/resnet50-imagenet.pth'
SNAPSHOT_DIR = 'snapshots/baseline_results/'

def str2bool(v):
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')


def get_arguments():
    """Parse all the arguments provided from the CLI.

    Returns:
      A list of parsed arguments.
    """
    parser = argparse.ArgumentParser(description="DeepLab-ResNet Network")

    parser.add_argument("--data-dir", type=str, default=DATA_DIRECTORY, help="Path to the directory containing the PASCAL VOC dataset.")
    parser.add_argument("--data-list", type=str, default=DATA_LIST_PATH, help="Path to the file listing the images in the dataset.")
    parser.add_argument("--ignore-label", type=int, default=IGNORE_LABEL, help="The index of the label to ignore during the training.")
    parser.add_argument("--input-size", type=str, default=INPUT_SIZE, help="Comma-separated string with height and width of images.")

    parser.add_argument("--is-training", action="store_true", help="Whether to updates the running means and variances during the training.")
    parser.add_argument("--batch-size", type=int, default=BATCH_SIZE, help="Number of images sent to the network in one step.")
    parser.add_argument("--learning-rate", type=float, default=LEARNING_RATE, help="Base learning rate for training with polynomial decay.")
    parser.add_argument("--momentum", type=float, default=MOMENTUM, help="Momentum component of the optimiser.")
    parser.add_argument("--not-restore-last", action="store_true", help="Whether to not restore last (FC) layers.")
    parser.add_argument("--start-iters", type=int, default=0, help="Number of classes to predict (including background).")
    parser.add_argument("--num-steps", type=int, default=NUM_STEPS, help="Number of training steps.")
    parser.add_argument("--power", type=float, default=POWER, help="Decay parameter to compute the learning rate.")
    parser.add_argument("--weight-decay", type=float, default=WEIGHT_DECAY, help="Regularisation parameter for L2-loss.")
    parser.add_argument("--gpu", type=str, default=GPU_NUM, help="choose gpu device.")
    parser.add_argument("--restore-from", type=str, default=RESTORE_FROM, help="Where restore model parameters from.")
    parser.add_argument("--save-num-images", type=int, default=SAVE_NUM_IMAGES, help="How many images to save.")
    parser.add_argument("--save-pred-every", type=int, default=SAVE_PRED_EVERY, help="Save summaries and checkpoint every often.")
    parser.add_argument("--snapshot-dir", type=str, default=SNAPSHOT_DIR, help="Where to save snapshots of the model.")

    parser.add_argument("--num-classes", type=int, default=NUM_CLASSES, help="Number of classes to predict (including background).")
    parser.add_argument("--random-mirror", action="store_true", help="Whether to randomly mirror the inputs during the training.")
    parser.add_argument("--random-scale", action="store_true", help="Whether to randomly scale the inputs during the training.")
    parser.add_argument("--random-seed", type=int, default=RANDOM_SEED, help="Random seed to have reproducible results.")
    parser.add_argument("--ft", type=bool, default=False, help="fine-tune the model with large input size.")
    parser.add_argument("--ohem", type=str2bool, default='False', help="use hard negative mining")
    parser.add_argument("--ohem-thres", type=float, default=0.6, help="choose the samples with correct probability under the threshold.")
    parser.add_argument("--ohem-keep", type=int, default=200000, help="choose the samples with correct probability under the threshold.")
    return parser.parse_args()

# args = get_arguments()

args = {'DATA_DIRECTORY':'/workspace/dataset/data',
       
        'BATCH_SIZE' : 6,
        'LEARNING_RATE' : 1e-2,
        'WEIGHT_DECAY' : 0.0005,

        'GPU_NUM' : '0',
        'INPUT_SIZE' : '512,512',

        'POWER' : 0.9,
        'MOMENTUM' : 0.9,
        'NUM_CLASSES' : 1,
        'NUM_STEPS' : 80000,
        'RANDOM_SEED' : 111,

        'SAVE_NUM_IMAGES' : 2,
        'SAVE_PRED_EVERY' : 5000,

        'RESTORE_FROM' : '/workspace/pretrain/resnet50-imagenet.pth',
        'SNAPSHOT_DIR' : 'snapshots/baseline_results/'
       }
args = {'data-dir':'workspace/dataset/data',
       
        'batch-size' : 6,
        'learning-rate' : 1e-2,
        'weight-decay' : 0.0005,

        'gpu' : '0',
        'input-size' : '512,512',

        'power' : 0.9,
        'momentum' : 0.9,
        'num-classes' : 1,
        'num-steps' : 80000,
        'random-seed' : 111,

        'save-num-images' : 2,
        'save-pred-every' : 5000,

        'restore-from' : 'workspace/pretrain/resnet50-imagenet.pth',
        'snapshot-dir' : 'snapshots/baseline_results/'
       }
print('here')

def lr_poly(base_lr, iter, max_iter, power):
    return base_lr * ((1 - float(iter) / max_iter) ** (power))


def adjust_learning_rate(optimizer, i_iter):
    """Sets the learning rate to the initial LR divided by 5 at 60th, 120th and 160th epochs"""
    lr = lr_poly(args.learning_rate, i_iter, args.num_steps, args.power)
    optimizer.param_groups[0]['lr'] = lr
    return lr


def set_bn_eval(m):
    classname = m.__class__.__name__
    if classname.find('BatchNorm') != -1:
        m.eval()


def set_bn_momentum(m):
    classname = m.__class__.__name__
    if classname.find('BatchNorm') != -1 or classname.find('InPlaceABN') != -1:
        m.momentum = 0.0003


def eval(testloader, model, input_size):
    average_meter = AverageMeter()
    model.eval()

    for index, batch in enumerate(tqdm(testloader)):
        image, label, ndsm, size, name = batch

        with torch.no_grad():
            output = predict_sliding(model, image.numpy(), input_size, 1, True)
        output = output * 183.17412
        seg_pred = output
        seg_pred = np.squeeze(seg_pred)

        dsm = np.array(ndsm)
        dsm = np.squeeze(dsm)
        result = Result()
        result.evaluate(seg_pred, dsm)
        average_meter.update(result, image.size(0))

    avg = average_meter.average()

    print('----------------------------------------------')
    print('[Eval]: DSM Estimate metrics:')
    print('==============================================')
    print('MAE={average.mae:.3f},    MSE={average.mse:.3f},    RMSE={average.rmse:.3f}\n'
          'Delta1={average.delta1:.3f}, Delta2={average.delta2:.3f}, Delta3={average.delta3:.3f}'.format(average=avg))
    print('==============================================')
    print('----------------------------------------------')



here


In [12]:
args

{'data-dir': 'workspace/dataset/data',
 'batch-size': 6,
 'learning-rate': 0.01,
 'weight-decay': 0.0005,
 'gpu': '0',
 'input-size': '512,512',
 'power': 0.9,
 'momentum': 0.9,
 'num-classes': 1,
 'num-steps': 80000,
 'random-seed': 111,
 'save-num-images': 2,
 'save-pred-every': 5000,
 'restore-from': 'workspace/pretrain/resnet50-imagenet.pth',
 'snapshot-dir': 'snapshots/baseline_results/'}

In [25]:
import sys
print(sys.argv)

['C:\\Users\\saada\\anaconda3\\envs\\pytorch\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\saada\\AppData\\Roaming\\jupyter\\runtime\\kernel-27c89143-498e-457e-b581-4b7570b609ea.json']


In [16]:

def main():
    """Create the model and start the training."""
    # writer = SummaryWriter(args.snapshot_dir)

    if not args.gpu == 'None':
        os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu
    h, w = map(int, args.input_size.split(','))
    input_size = (h, w)
    cudnn.enabled = True
    # -----------------------------
    # Create network.
    # -----------------------------
    net = Res_Deeplab(num_classes=args.num_classes)
    saved_state_dict = torch.load(args.restore_from, map_location='cpu')
    new_params = net.state_dict().copy()

    for i in saved_state_dict:
        i_parts = i.split('.')
        if not i_parts[0] == 'fc':
            new_params['.'.join(i_parts[0:])] = saved_state_dict[i]
    net.load_state_dict(new_params)
    model = DataParallelModel(net)
    model.cuda()

    if args.ohem:
        criterion = CriterionOhemDSN(thresh=args.ohem_thres, min_kept=args.ohem_keep)
    else:
        criterion = CriterionDSN()

    criterion = DataParallelCriterion(criterion)
    cudnn.benchmark = True

    if not os.path.exists(args.snapshot_dir):
        os.makedirs(args.snapshot_dir)
    traindataset = ISPRSDataSet(args.data_dir, args.data_list,
                                max_iters=args.num_steps * args.batch_size,
                         crop_size=input_size, scale=args.random_scale,
                                mirror=args.random_mirror, mean=IMG_MEAN)
    trainloader = data.DataLoader(traindataset, batch_size=args.batch_size, shuffle=True, num_workers=0, pin_memory=True)

    testdataset = ISPRSDataValSet(args.data_dir, TEST_DATA_LIST_PATH, mean=IMG_MEAN, scale=False, mirror=False)
    testloader = data.DataLoader(testdataset,batch_size=1, shuffle=False, pin_memory=True)

    optimizer = optim.SGD([{'params': filter(lambda p: p.requires_grad, net.parameters()), 'lr': args.learning_rate}],
                          lr=args.learning_rate, momentum=args.momentum, weight_decay=args.weight_decay)
    optimizer.zero_grad()

    for i_iter, batch in enumerate(trainloader):
        i_iter += args.start_iters

        images, dsms, labels, size, _ = batch

        images = images.cuda()
        labels = labels.long().cuda()
        dsms = dsms.cuda()

        model.train()
        model.float()

        if torch_ver == "0.3":
            images = Variable(images)
            labels = Variable(labels)
            dsms = Variable(dsms)

        optimizer.zero_grad()
        lr = adjust_learning_rate(optimizer, i_iter)

        preds = model(images)
        pred_up = F.upsample(input=preds[0], size=(labels.size(1), labels.size(2)), mode='bilinear', align_corners=True)
        loss = criterion(pred_up, dsms)

        loss.backward()
        optimizer.step()

        if i_iter % 100 == 0:
            print('======[Train]: iter:{}/{} | loss = {} | lr = {}======'.format(i_iter, args.num_steps, loss.data.cpu().numpy(), lr))

        if i_iter >= args.num_steps - 1:
            print('================Save model====================')
            torch.save(net.state_dict(), osp.join(args.snapshot_dir, 'ISPRS_lr1e-2_' + str(args.num_steps) + '.pth'))
            break

        if i_iter!= 0 and i_iter % args.save_pred_every == 0:
            print('----------------------------------------------')
            print('===================Eval=======================')
            h, w = map(int, args.input_size.split(','))
            input_size = (h, w)
            eval(testloader, model, input_size)
            torch.save(net.state_dict(), osp.join(args.snapshot_dir, 'ISPRS_lr1e-2_' + str(i_iter) + '.pth'))



In [24]:
args = get_arguments()

usage: ipykernel_launcher.py [-h] [--data-dir DATA_DIR]
                             [--data-list DATA_LIST]
                             [--ignore-label IGNORE_LABEL]
                             [--input-size INPUT_SIZE] [--is-training]
                             [--batch-size BATCH_SIZE]
                             [--learning-rate LEARNING_RATE]
                             [--momentum MOMENTUM] [--not-restore-last]
                             [--start-iters START_ITERS]
                             [--num-steps NUM_STEPS] [--power POWER]
                             [--weight-decay WEIGHT_DECAY] [--gpu GPU]
                             [--restore-from RESTORE_FROM]
                             [--save-num-images SAVE_NUM_IMAGES]
                             [--save-pred-every SAVE_PRED_EVERY]
                             [--snapshot-dir SNAPSHOT_DIR]
                             [--num-classes NUM_CLASSES] [--random-mirror]
                             [--random-scale] [--rand

SystemExit: 2

In [18]:
!python 

python: can't open file 'get_arguments()': [Errno 2] No such file or directory


In [17]:

if __name__ == '__main__':
    main()


AttributeError: 'dict' object has no attribute 'gpu'

In [3]:
torch.__version__



NameError: name 'torch' is not defined

In [4]:
import torch
!python --version


C:\Users\saada\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Python 3.7.15


In [19]:
!conda upgrade -c conda-forge open-cv



PackageNotInstalledError: Package is not installed in prefix.
  prefix: C:\Users\saada\anaconda3\envs\pytorch
  package name: open-cv




In [5]:
import cv2 
print(cv2.__version__)


4.6.0


In [23]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Jun__8_16:59:34_Pacific_Daylight_Time_2022
Cuda compilation tools, release 11.7, V11.7.99
Build cuda_11.7.r11.7/compiler.31442593_0


In [24]:
import json

# Opening JSON file
f = open(r'C:\Users\saada\Desktop\Drive\UBS\Workshop_Contest\data\track2\buildings_only_train.json',)

# returns JSON object as
# a dictionary
data = json.load(f)

# Iterating through the json
# list
with open('train_imgs.txt', 'w') as f:
    for i in data['images']:
        print(i['file_name'].split('.ti')[0])
        f.write("%s\n" % i['file_name'].split('.ti')[0])
        

    

GF2_Brasilia_-15.8652_-47.9337
GF2_Brasilia_-15.8652_-47.9351
GF2_Brasilia_-15.8652_-47.9366
GF2_Brasilia_-15.8652_-47.9381
GF2_Brasilia_-15.8653_-47.9277
GF2_Brasilia_-15.8653_-47.9279
GF2_Brasilia_-15.8653_-47.9293
GF2_Brasilia_-15.8653_-47.9308
GF2_Brasilia_-15.8653_-47.9322
GF2_Brasilia_-15.8666_-47.9337
GF2_Brasilia_-15.8666_-47.9352
GF2_Brasilia_-15.8666_-47.9366
GF2_Brasilia_-15.8666_-47.9381
GF2_Brasilia_-15.8666_-47.9395
GF2_Brasilia_-15.8667_-47.9277
GF2_Brasilia_-15.8667_-47.9279
GF2_Brasilia_-15.8667_-47.9293
GF2_Brasilia_-15.8667_-47.9308
GF2_Brasilia_-15.8667_-47.9323
GF2_Brasilia_-15.8680_-47.9352
GF2_Brasilia_-15.8680_-47.9366
GF2_Brasilia_-15.8680_-47.9381
GF2_Brasilia_-15.8680_-47.9395
GF2_Brasilia_-15.8681_-47.9277
GF2_Brasilia_-15.8681_-47.9279
GF2_Brasilia_-15.8681_-47.9294
GF2_Brasilia_-15.8681_-47.9308
GF2_Brasilia_-15.8681_-47.9323
GF2_Brasilia_-15.8681_-47.9337
GF2_Brasilia_-15.8694_-47.9352
GF2_Brasilia_-15.8694_-47.9367
GF2_Brasilia_-15.8694_-47.9381
GF2_Bras

'SV_Sydney_-33.8688_151.1954'

In [86]:

# Iterating through the json
# list
for i in data['annotations']:
    print(i)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{'segmentation': [[511, 135, 510, 136, 510, 137, 508, 139, 508, 140, 506, 142, 506, 143, 504, 145, 504, 146, 502, 148, 502, 149, 500, 151, 500, 152, 498, 154, 498, 155, 497, 156, 497, 157, 495, 159, 495, 160, 493, 162, 493, 163, 491, 165, 491, 166, 489, 168, 489, 169, 487, 171, 487, 172, 485, 174, 485, 175, 483, 177, 483, 178, 481, 180, 481, 181, 479, 183, 479, 184, 477, 186, 477, 187, 476, 188, 476, 189, 474, 191, 474, 192, 472, 194, 472, 195, 470, 197, 470, 198, 468, 200, 468, 201, 466, 203, 466, 204, 464, 206, 464, 207, 462, 209, 462, 210, 460, 212, 460, 213, 458, 215, 458, 216, 456, 218, 456, 219, 455, 220, 455, 221, 453, 223, 453, 224, 451, 226, 451, 227, 449, 229, 449, 230, 447, 232, 447, 233, 445, 235, 445, 236, 443, 238, 443, 239, 441, 241, 441, 242, 439, 244, 439, 245, 437, 247, 437, 248, 435, 250, 435, 251, 434, 252, 434, 253, 432, 255, 432, 256, 430, 258, 430, 259, 428, 261, 428, 262, 426, 264, 426, 265, 424, 267, 424, 268, 422, 270, 422, 271, 420, 273, 420, 274, 418, 276, 4

In [88]:

# Iterating through the json
# list
for i in data['categories']:
    print(i)

{'supercategory': 'building', 'id': 1, 'name': 'building'}


In [ ]:
# !python main.py --model S2ENet --flip_augmentation --patch_size 7 --epoch 128 --lr 0.001 --batch_size 64 --seed 0 --dataset Houston2013 --folder '../' --train_set '../Houston2013/TRLabel.mat' --test_set '../Houston2013/TSLabel.mat' --cuda 0

In [1]:
import json

# Opening JSON file
f = open(r'C:\Users\saada\Desktop\Drive\UBS\Workshop_Contest\data\track2\buildings_only_train.json',)

# returns JSON object as
# a dictionary
data = json.load(f)

# Iterating through the json
# list
with open('train_imgs.txt', 'w') as f:
    for i in data['images']:
        print(i['file_name'].split('.ti')[0])
        #f.write("%s\n" % i['file_name'].split('.ti')[0])
        

    

GF2_Brasilia_-15.8652_-47.9337
GF2_Brasilia_-15.8652_-47.9351
GF2_Brasilia_-15.8652_-47.9366
GF2_Brasilia_-15.8652_-47.9381
GF2_Brasilia_-15.8653_-47.9277
GF2_Brasilia_-15.8653_-47.9279
GF2_Brasilia_-15.8653_-47.9293
GF2_Brasilia_-15.8653_-47.9308
GF2_Brasilia_-15.8653_-47.9322
GF2_Brasilia_-15.8666_-47.9337
GF2_Brasilia_-15.8666_-47.9352
GF2_Brasilia_-15.8666_-47.9366
GF2_Brasilia_-15.8666_-47.9381
GF2_Brasilia_-15.8666_-47.9395
GF2_Brasilia_-15.8667_-47.9277
GF2_Brasilia_-15.8667_-47.9279
GF2_Brasilia_-15.8667_-47.9293
GF2_Brasilia_-15.8667_-47.9308
GF2_Brasilia_-15.8667_-47.9323
GF2_Brasilia_-15.8680_-47.9352
GF2_Brasilia_-15.8680_-47.9366
GF2_Brasilia_-15.8680_-47.9381
GF2_Brasilia_-15.8680_-47.9395
GF2_Brasilia_-15.8681_-47.9277
GF2_Brasilia_-15.8681_-47.9279
GF2_Brasilia_-15.8681_-47.9294
GF2_Brasilia_-15.8681_-47.9308
GF2_Brasilia_-15.8681_-47.9323
GF2_Brasilia_-15.8681_-47.9337
GF2_Brasilia_-15.8694_-47.9352
GF2_Brasilia_-15.8694_-47.9367
GF2_Brasilia_-15.8694_-47.9381
GF2_Bras

In [17]:
import glob

path= r'C:\Users\saada\Desktop\Drive\UBS\Workshop_Contest\track2\HeightEstimation\workspace\dataset\data\val\rgb/'

glob.glob(path+"*")


with open('val_imgs.txt', 'w') as f:
    for i in glob.glob(path+"*"):
        g = i.split('rgb\P')[1].split('.')[0]
        g='P'+g
        print(g)

        f.write("%s\n" % g)

P_0000
P_0001
P_0002
P_0003
P_0004
P_0005
P_0006
P_0007
P_0008
P_0009
P_0010
P_0011
P_0012
P_0013
P_0014
P_0015
P_0016
P_0017
P_0018
P_0019
P_0020
P_0021
P_0022
P_0023
P_0024
P_0025
P_0026
P_0027
P_0028
P_0029
P_0030
P_0031
P_0032
P_0033
P_0034
P_0035
P_0036
P_0037
P_0038
P_0039
P_0040
P_0041
P_0042
P_0043
P_0044
P_0045
P_0046
P_0047
P_0048
P_0049
P_0050
P_0051
P_0052
P_0053
P_0054
P_0055
P_0056
P_0057
P_0058
P_0059
P_0060
P_0061
P_0062
P_0063
P_0064
P_0065
P_0066
P_0067
P_0068
P_0069
P_0070
P_0071
P_0072
P_0073
P_0074
P_0075
P_0076
P_0077
P_0078
P_0079
P_0080
P_0081
P_0082
P_0083
P_0084
P_0085
P_0086
P_0087
P_0088
P_0089
P_0090
P_0091
P_0092
P_0093
P_0094
P_0095
P_0096
P_0097
P_0098
P_0099
P_0100
P_0101
P_0102
P_0103
P_0104
P_0105
P_0106
P_0107
P_0108
P_0109
P_0110
P_0111
P_0112
P_0113
P_0114
P_0115
P_0116
P_0117
P_0118
P_0119
P_0120
P_0121
P_0122
P_0123
P_0124
P_0125
P_0126
P_0127
P_0128
P_0129
P_0130
P_0131
P_0132
P_0133
P_0134
P_0135
P_0136
P_0137
P_0138
P_0139
P_0140
P_0141
P_0142

In [1]:
import glob

path= r'C:\Users\saada\Desktop\Drive\UBS\Workshop_Contest\track2\HeightEstimation\workspace\dataset\data\track2_test_data\rgb/'

glob.glob(path+"*")


with open('test.txt', 'w') as f:
    for i in glob.glob(path+"*"):
        g = i.split('rgb\P')[1].split('.')[0]
        g='P'+g
        print(g)

        f.write("%s\n" % g)

P_0420
P_0421
P_0422
P_0423
P_0424
P_0425
P_0426
P_0427
P_0428
P_0429
P_0430
P_0431
P_0432
P_0433
P_0434
P_0435
P_0436
P_0437
P_0438
P_0439
P_0440
P_0441
P_0442
P_0443
P_0444
P_0445
P_0446
P_0447
P_0448
P_0449
P_0450
P_0451
P_0452
P_0453
P_0454
P_0455
P_0456
P_0457
P_0458
P_0459
P_0460
P_0461
P_0462
P_0463
P_0464
P_0465
P_0466
P_0467
P_0468
P_0469
P_0470
P_0471
P_0472
P_0473
P_0474
P_0475
P_0476
P_0477
P_0478
P_0479
P_0480
P_0481
P_0482
P_0483
P_0484
P_0485
P_0486
P_0487
P_0488
P_0489
P_0490
P_0491
P_0492
P_0493
P_0494
P_0495
P_0496
P_0497
P_0498
P_0499
P_0500
P_0501
P_0502
P_0503
P_0504
P_0505
P_0506
P_0507
P_0508
P_0509
P_0510
P_0511
P_0512
P_0513
P_0514
P_0515
P_0516
P_0517
P_0518
P_0519
P_0520
P_0521
P_0522
P_0523
P_0524
P_0525
P_0526
P_0527
P_0528
P_0529
P_0530
P_0531
P_0532
P_0533
P_0534
P_0535
P_0536
P_0537
P_0538
P_0539
P_0540
P_0541
P_0542
P_0543
P_0544
P_0545
P_0546
P_0547
P_0548
P_0549
P_0550
P_0551
P_0552
P_0553
P_0554
P_0555
P_0556
P_0557
P_0558
P_0559
P_0560
P_0561
P_0562